In [1]:
import spacy
nlp = spacy.load('en')
doc = nlp('Mission Impossible was written by Chrisopher Nolan.')

In [4]:
doc = nlp('Mission Impossible was written by Chrisopher Nolan.')
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Chrisopher Nolan 34 50 ORG


In [160]:
#statement1 = 'The Mission Impossible was written by Chrisopher Nolan.'
statement1 = 'The Searching is directed by Aneesh Chaganty in his feature debut and written by Chaganty and Sev Ohanian'
#statement1 = 'herry film was written and produced by Joss Whedon and directed by jorge.'
statement2 = 'Chrisopher Nolan wrote Mission Impossible.'
isValidForWrittenTemplate = checkWrittenTemplateEligibility(statement1)
if(isValidForWrittenTemplate):
#     time = extractReleaseTime(statement)
#     print("extracted time is:"+str(time))
    
    movieName = extractMovieName(statement1)
    print("extracted movie name is:" + str(movieName))
    
    writerName = extractWriterName(statement1)
    print("extracted writer name is:" + str(writerName))

Matching with wriiten template: for The film is directed by Aneesh Chaganty in his feature debut and written by Chaganty and Sev Ohanian
Has valid verb:True
vt:written
token conj : written
token.head: directed
token nsubjpass : film
extracted movie name is:The film
verbTextChanged todirected
vt:directed
extracted writer name is:Aneesh Chaganty


In [126]:
def extractMovieName(statement):
    targetVerbLemmas = ['write']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    movieName = "<uninitialized>"
    
#     handle when release is with premiered.
    '''
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to : "+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    '''

    print("vt:"+verbText)

    movieName = getMovie(statement,verbText)
    
    return movieName

In [59]:
def extractWriterName(statement):
    targetVerbLemmas = ['write']
    verbText = getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas)
    verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)
    writerName = "<uninitialized>"
    
#     handle when release is with premiered.
    while verbTextDNode[1] == 'conj':
        verbText = verbTextDNode[2]
        print("verbTextChanged to"+verbText)
        verbTextDNode = next(obj for obj in get_dependency_tree_nodes(statement) if obj[0] == verbText)

    print("vt:"+verbText)

    writerName = getWriter(statement,verbText)
    
    return writerName

In [33]:
def getTargetVerbAsAppearedInStatement(statement, targetVerbLemmas):
    """
    this will reutrn exact apperance of verb in statement.
    ex: statement = "Movie was written by James.", targetVerbLemmas = ['write','script']
    this will return "written" as it's lemma matches with one of the targetVerbLemmas
    """
    verbTokens = filterVerbTokens(statement)
    for token in verbTokens:
        if token.lemma_ in targetVerbLemmas:
            return token.text
    return "<no-match>"

In [42]:
def getNounSubject(statement, verbText):
    doc = nlp(statement)
    for token in doc:
        print(token.text, token.dep_, token.head.text)
        if((token.dep_ == "nsubjpass" or token.dep_ == "nsubj") and token.head.text == verbText):
            return token.text
    return "<no-match>"

In [154]:
def getMovie(statement, verbText):
    doc = nlp(statement)
    theme = "<uninitialized>" #movie
    
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "nsubjpass" and token.head.text == verbText):
            print("token nsubjpass : " + token.text)
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif (token.dep_ == "dobj" and token.head.text == verbText):
            theme = getNounChunkThatContainsNoun(statement, token)
            return theme
        elif ((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            print("token conj : " + token.text)
            if (token.head.dep_ == "ROOT"):
                print("token.head: " + token.head.text)
                token1 = token.head
                return getMovie(statement, token1.text)
                '''
                if (token.dep_ == "nsubjpass" and token.head.text == token1.text):
                    theme = getNounChunkThatContainsNoun(statement, token)
                    return theme
                '''
    
    return "<no-match>"

In [143]:
def getWriter(statement, verbText):
    doc = nlp(statement)
    agent = "<uninitialized>" #writer
    '''
    for token in doc:
        #print(token.text, token.dep_, token.head.text)        
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
    '''
    
    for token in doc:
        #print("token:",token)
        if (token.dep_ == "agent" and token.head.text == verbText):
            token = [child for child in token.children][0]
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        elif (token.dep_ == "nsubj" and token.head.text == verbText):
            agent = getNounChunkThatContainsNoun(statement, token)
            return agent
        
        elif((token.dep_ == "conj" and token.head.text == verbText) or (token.dep_ == "conj" and token.text == verbText)):
            #print("conj: "+ token.text)
            token1 = [child for child in token.children]
            #print(token1[0], token1[1])
            i = 0
            while (i < len(token1)):
                if (token1[i].dep_ == "agent"):
                    token2 = [child1 for child1 in token1[i].children][0]
                    #print(token2)
                    agent = getNounChunkThatContainsNoun(statement, token2)
                    return agent
                i+=1
        
    return agent

In [55]:
def getNounChunkThatContainsNoun(statement, nounSubject):
    doc = nlp(statement)
    for chunk in doc.noun_chunks:
        if nounSubject.text in chunk.text:
            return chunk.text
    return "<no-match>"

In [36]:
def checkWrittenTemplateEligibility(statement):
    print("Matching with wriiten template: for " + statement)
    hasValidVerb = check_verb_match(statement)
    print("Has valid verb:" + str(hasValidVerb))
    
    return hasValidVerb

In [37]:
def check_verb_match(statement):
    verbTokens = filterVerbTokens(statement)
    targetVerbs = ['write']
    for token in verbTokens:
        if token.lemma_ in targetVerbs:
            return True
    return False

In [38]:
def filterVerbTokens(statement):
    doc = nlp(statement)
    result = list(filter(lambda token: token.pos_ == "VERB" and token.lemma_ != "be", doc))
    return result

In [14]:
def get_dependency_tree_nodes(sentence):
    nodes = []
    doc = nlp(sentence)
    for token in doc:
        nodes.append([token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children]])
    return nodes

In [46]:
get_dependency_tree_nodes('The Mission Impossible was written by Chrisopher Nolan.')

[['The', 'det', 'Impossible', 'PROPN', []],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'nsubjpass', 'written', 'VERB', [The, Mission]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [Impossible, was, by, .]],
 ['by', 'agent', 'written', 'VERB', [Nolan]],
 ['Chrisopher', 'compound', 'Nolan', 'PROPN', []],
 ['Nolan', 'pobj', 'by', 'ADP', [Chrisopher]],
 ['.', 'punct', 'written', 'VERB', []]]

In [16]:
get_dependency_tree_nodes('Chrisopher Nolan wrote Mission Impossible.')

[['Chrisopher', 'compound', 'Nolan', 'PROPN', []],
 ['Nolan', 'nsubj', 'wrote', 'VERB', [Chrisopher]],
 ['wrote', 'ROOT', 'wrote', 'VERB', [Nolan, Impossible, .]],
 ['Mission', 'compound', 'Impossible', 'PROPN', []],
 ['Impossible', 'dobj', 'wrote', 'VERB', [Mission]],
 ['.', 'punct', 'wrote', 'VERB', []]]

In [162]:
get_dependency_tree_nodes('The Searching is directed by Aneesh Chaganty in his feature debut and written by Chaganty and Sev Ohanian')

[['The', 'det', 'Searching', 'PROPN', []],
 ['Searching', 'nsubjpass', 'directed', 'VERB', [The]],
 ['is', 'auxpass', 'directed', 'VERB', []],
 ['directed',
  'ROOT',
  'directed',
  'VERB',
  [Searching, is, by, in, and, written]],
 ['by', 'agent', 'directed', 'VERB', [Chaganty]],
 ['Aneesh', 'compound', 'Chaganty', 'PROPN', []],
 ['Chaganty', 'pobj', 'by', 'ADP', [Aneesh]],
 ['in', 'prep', 'directed', 'VERB', [debut]],
 ['his', 'poss', 'debut', 'NOUN', []],
 ['feature', 'compound', 'debut', 'NOUN', []],
 ['debut', 'pobj', 'in', 'ADP', [his, feature]],
 ['and', 'cc', 'directed', 'VERB', []],
 ['written', 'conj', 'directed', 'VERB', [by]],
 ['by', 'agent', 'written', 'VERB', [Chaganty]],
 ['Chaganty', 'pobj', 'by', 'ADP', [and, Sev, Ohanian]],
 ['and', 'cc', 'Chaganty', 'PROPN', []],
 ['Sev', 'conj', 'Chaganty', 'PROPN', []],
 ['Ohanian', 'conj', 'Chaganty', 'PROPN', []]]

In [74]:
get_dependency_tree_nodes('herry film was written and directed by Joss Whedon')

[['herry', 'amod', 'film', 'NOUN', []],
 ['film', 'nsubjpass', 'written', 'VERB', [herry]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [film, was, and, directed]],
 ['and', 'cc', 'written', 'VERB', []],
 ['directed', 'conj', 'written', 'VERB', [by]],
 ['by', 'agent', 'directed', 'VERB', [Whedon]],
 ['Joss', 'compound', 'Whedon', 'PROPN', []],
 ['Whedon', 'pobj', 'by', 'ADP', [Joss]]]

In [107]:
get_dependency_tree_nodes('herry film was written and produced by Joss Whedon and directed by jorge.')

[['herry', 'amod', 'film', 'NOUN', []],
 ['film', 'nsubjpass', 'written', 'VERB', [herry]],
 ['was', 'auxpass', 'written', 'VERB', []],
 ['written', 'ROOT', 'written', 'VERB', [film, was, and, produced, .]],
 ['and', 'cc', 'written', 'VERB', []],
 ['produced', 'conj', 'written', 'VERB', [by, and, directed]],
 ['by', 'agent', 'produced', 'VERB', [Whedon]],
 ['Joss', 'compound', 'Whedon', 'PROPN', []],
 ['Whedon', 'pobj', 'by', 'ADP', [Joss]],
 ['and', 'cc', 'produced', 'VERB', []],
 ['directed', 'conj', 'produced', 'VERB', [by]],
 ['by', 'agent', 'directed', 'VERB', [jorge]],
 ['jorge', 'pobj', 'by', 'ADP', []],
 ['.', 'punct', 'written', 'VERB', []]]

In [156]:
getNounChunkThatContainsNoun('Searching is directed by Aneesh Chaganty in his feature debut and written by Chaganty and Sev Ohanian', Searching)

NameError: name 'Searching' is not defined

In [159]:
doc = nlp("Dark Knight is directed by Aneesh Chaganty in his feature debut and written by Chaganty and Sev Ohanian")
for chunk in doc.noun_chunks:
    print(chunk)

Dark Knight
Aneesh Chaganty
his feature debut
Chaganty
Sev
Ohanian
